In [1]:
from src.contracts import *

Consider a contract for bounding a battery charging behavior where `soc*` variables represent the battery state-of-charge, `t*` variables represent time, `duration*` represents the duration of the contract between `t*_entry` and `t*_exit`. The `s` index allwos for creating distinct copies of this contract for modeling distinct occurrences.

In [2]:
c1: PolyhedralContract = charging(s=1, generation=[0.5, 0.8])
print(f"c1:\n{c1}")

c1:
InVars: [t1_entry, t1_var, duration1, soc1_entry]
OutVars:[t1_exit, soc1_var, soc1_exit]
A: [
  -duration1 <= -0
  -t1_entry <= -0
  t1_entry - t1_var <= 0
]
G: [
  -soc1_entry <= 0
  0.8 duration1 + soc1_entry <= 100
  duration1 + t1_entry - t1_exit = 0
  0.5 duration1 + soc1_entry - soc1_exit <= 0
  -0.8 duration1 - soc1_entry + soc1_exit <= 0
  soc1_entry - soc1_var - 0.5 t1_entry + 0.5 t1_var <= 0
  -soc1_entry + soc1_var + 0.8 t1_entry - 0.8 t1_var <= 0
  -t1_exit + t1_var <= 0
  -soc1_exit + soc1_var <= 0
]


To use a contract for simulation purposes, we need to extract its `PolyhedralTermList`: the list of all assumptions and guarantees constraints.

In [3]:
constraints: PolyhedralTermList = c1.a | c1.g
print(f"constraints:\n{constraints}")

constraints:
[
  -duration1 <= -0
  -t1_entry <= -0
  t1_entry - t1_var <= 0
  -soc1_entry <= 0
  0.8 duration1 + soc1_entry <= 100
  duration1 + t1_entry - t1_exit = 0
  0.5 duration1 + soc1_entry - soc1_exit <= 0
  -0.8 duration1 - soc1_entry + soc1_exit <= 0
  soc1_entry - soc1_var - 0.5 t1_entry + 0.5 t1_var <= 0
  -soc1_entry + soc1_var + 0.8 t1_entry - 0.8 t1_var <= 0
  -t1_exit + t1_var <= 0
  -soc1_exit + soc1_var <= 0
]


We can refine this list of constraints in a particular context of values for the exogeneous variables; which is a subset of the contract's input variables.

In [7]:
ce: PolyhedralTermList = constraints.evaluate({
    Var("duration1"): 10.0,
    Var("t1_entry"): 0,
    Var("soc1_entry"): 50.0,
}).simplify()
print(f"ce:\n{ce}")

ce:
[
  -t1_exit = -10
  -soc1_exit <= -55
  soc1_exit <= 58
  -soc1_var + 0.5 t1_var <= -50
  soc1_var - 0.8 t1_var <= 50
  -t1_exit + t1_var <= 0
  -soc1_exit + soc1_var <= 0
]


With these refined constraints, we can compute bounds for the simulation input, `t1_var`, and output, `soc1_var`.

In [8]:
t_var_min, t_var_max = get_bounds(ce, "t1_var")
print(f"t1_var:[{t_var_min},{t_var_max}]")

soc1_var_min, soc1_var_max = get_bounds(ce, "soc1_var")
print(f"soc1_var (ce) :[{soc1_var_min},{soc1_var_max}]")

t1_var:[0.0,10.0]
soc1_var (ce) :[50.0,58.0]


We can evaluate the refined constraints for a specific value of the simulation input, `t1_var`, to get bounds on the simulation output, `soc1_var`.

In [9]:
ce1: PolyhedralTermList = ce.evaluate({Var("t1_var"): 2.0}).simplify()
print(f"ce1:\n{ce1}")
soc1_var_min, soc1_var_max = get_bounds(ce1, "soc1_var")
print(f"soc1_var (ce1):[{soc1_var_min},{soc1_var_max}]")

ce1:
[
  -t1_exit = -10
  -soc1_exit <= -55
  soc1_exit <= 58
  -soc1_var <= -51
  soc1_var <= 51.6
]
soc1_var (ce1):[51.0,51.6]
